In [1]:
from __future__ import print_function

import os

import numpy as np

np.random.seed(123)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Embedding, Convolution1D, MaxPooling1D, Flatten
from keras.layers import Input, Merge, Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

databaseConnectionServer = 'srn02.cs.cityu.edu.hk'
documentTable = 'document'

BASE_DIR = '../../'
GLOVE_DIR = BASE_DIR + 'glove/'
EMBEDDING_DIM = 200
embedfile = 'glove.6B.' + str(EMBEDDING_DIM) + 'd.txt'

chunk_size = 1000
samples = 3200

MAX_NB_WORDS = 40000
VALIDATION_SPLIT = 0.2

MAX_NB_WORDS = 40000

CONVOLUTION_FEATURE = 256
BORDER_MODE = 'valid'
DENSE_FEATURE = 256
DROP_OUT = 0.5
LEARNING_RATE=0.01
MOMENTUM=0.9

nb_epoch=10
batch_size=16

doc_id = 85
author_id = 44
authorList = [64,44,82]

MAX_SEQUENCE_LENGTH = chunk_size

classes = len(authorList)

Using TensorFlow backend.


In [2]:
print('Level = Word')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, embedfile))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('File used: %s' % (embedfile))
print('Found %s word vectors.' % (len(embeddings_index)))

Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.


In [3]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
import DatabaseQuery
from sshtunnel import SSHTunnelForwarder
PORT=5432
with SSHTunnelForwarder((databaseConnectionServer, 22),
                        ssh_username='stylometry',
                        ssh_password='stylometry',
                        remote_bind_address=('localhost', 5432),
                        local_bind_address=('localhost', 5400)):
    textToUse = DatabaseQuery.getWordAuthData(5400, authorList, doc_id,
                                              documentTable = documentTable, chunk_size = chunk_size)
labels = []
texts = []
size = []
authorList = textToUse.author_id.unique()
for auth in authorList:
    current = textToUse.loc[textToUse['author_id'] == auth]
    size.append(current.shape[0])
    print("Author: %5s  Size: %5s" % (auth, current.shape[0]))
print("Min: %s" % (min(size)))
print("Max: %s" % (max(size)))

authorList = authorList.tolist()

for auth in authorList:
    current = textToUse.loc[textToUse['author_id'] == auth]
    if (samples > min(size)):
        samples = min(size)
    current = current.sample(n = samples)
    textlist = current.doc_content.tolist()
    texts = texts + textlist
    labels = labels + [authorList.index(author_id) for author_id in current.author_id.tolist()]
labels_index = {}
labels_index[0] = 0
for i, auth in enumerate(authorList):
    labels_index[i] = auth

del textToUse

print('Authors %s.' % (str(authorList)))
print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels))

Execution completed
Read completed
Number of rows: 124
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (11578, 2)
Author:    44  Size:  4746
Author:    64  Size:  5106
Author:    82  Size:  1726
Min: 1726
Max: 5106
Authors [44, 64, 82].
Found 5178 texts.
Found 5178 labels.


In [4]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=chunk_size)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
from sklearn.model_selection import train_test_split
trainX, valX, trainY, valY = train_test_split(data, labels, test_size=VALIDATION_SPLIT)

del data, labels

Found 46523 unique tokens.
Shape of data tensor: (5178, 1000)
Shape of label tensor: (5178, 3)


In [5]:
print('Preparing embedding matrix.')

# prepare embedding matrix
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

del embeddings_index

print('Training model.')

Preparing embedding matrix.
Training model.


In [6]:
print(nb_words)
print(MAX_NB_WORDS)
print(len(word_index))

46523
40000
46523


In [7]:
# def getModel():
ngram_filters = [3, 4]                                  # Define ngrams list, 3-gram, 4-gram, 5-gram
convs = []

graph_in = Input(shape=(chunk_size, EMBEDDING_DIM))

for n_gram in ngram_filters:
    conv = Convolution1D(                                  # Layer X,   Features: 256, Kernel Size: ngram
        nb_filter=CONVOLUTION_FEATURE,                     # Number of kernels or number of filters to generate
        filter_length=n_gram,                              # Size of kernels, ngram
        activation='relu',                                 # Activation function to use
        name = str('convfilter' + str(n_gram)))(graph_in)

    pool = MaxPooling1D(                                   # Layer X a,  Max Pooling: 3
        pool_length=3)(conv)                               # Size of kernels

    flat = Flatten()(pool)

    convs.append(flat)

model = Sequential()

model.add(Embedding(                                       # Layer 0, Start
    input_dim=nb_words + 1,                                # Size to dictionary, has to be input + 1
    output_dim=EMBEDDING_DIM,                              # Dimensions to generate
    weights=[embedding_matrix],                            # Initialize word weights
    input_length=chunk_size,                               # Define length to input sequences in the first layer
    trainable=False))                                      # Disable weight changes during training

model.add(Dropout(0.25))                                   # Dropout 25%

out = Merge(mode='concat')(convs)                          # Layer 1,  Output Size: Concatted ngrams feature maps

graph = Model(input=graph_in, output=out, name = 'merge')  # Concat the ngram convolutions

model.add(graph)                                           # Concat the ngram convolutions

model.add(Dropout(DROP_OUT))                               # Dropout 50%

model.add(Dense(                                           # Layer 3,  Output Size: 256
    output_dim=DENSE_FEATURE,                              # Output dimension
    activation='relu'))                                    # Activation function to use

model.add(Dense(                                           # Layer 4,  Output Size: Size Unique Labels, Final
    output_dim=classes,                                    # Output dimension
    activation='softmax'))                                 # Activation function to use

sgd = SGD(lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])


In [8]:
filepath="author-cnn-ngrams-word.hdf5"
    
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

# Function to take input of data and return fitted model
history = model.fit(trainX, trainY, validation_data=(valX, valY),
                    nb_epoch=nb_epoch, batch_size=batch_size,
                    callbacks=callbacks_list)

# load weights from the best checkpoint
model.load_weights(filepath)

# Compile model again (required to make predictions)
model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])

train_acc = (model.evaluate(trainX, trainY))[1]
print("\n\nFinal Train Accuracy: %.2f" % (train_acc * 100))

val_acc = (model.evaluate(valX, valY))[1]
print("\nFinal Test Accuracy: %.2f" % (val_acc * 100))

import cPickle as pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Train on 4142 samples, validate on 1036 samples
Epoch 1/10
4142/4142 [==============================] - 35s - loss: 0.9091 - acc: 0.5633 - val_loss: 0.6536 - val_acc: 0.6670
Epoch 2/10
4142/4142 [==============================] - 32s - loss: 0.4357 - acc: 0.8279 - val_loss: 0.3478 - val_acc: 0.8822
Epoch 3/10
4142/4142 [==============================] - 32s - loss: 0.1956 - acc: 0.9259 - val_loss: 0.3083 - val_acc: 0.8871
Epoch 4/10
4142/4142 [==============================] - 32s - loss: 0.1109 - acc: 0.9619 - val_loss: 0.1979 - val_acc: 0.9286
Epoch 5/10
4142/4142 [==============================] - 31s - loss: 0.0688 - acc: 0.9737 - val_loss: 0.2927 - val_acc: 0.9093
Epoch 6/10
4142/4142 [==============================] - 32s - loss: 0.0718 - acc: 0.9739 - val_loss: 0.1813 - val_acc: 0.9431
Epoch 7/10
4142/4142 [==============================] - 32s - loss: 0.0475 - acc: 0.9809 - val_loss: 0.1796 - val_acc: 0.9440
Epoch 8/10
4142/4142 [==============================] - 32s - loss: 0.

In [9]:
# def getModel():
ngram_filters = [3, 4]                                  # Define ngrams list, 3-gram, 4-gram, 5-gram
convs = []

graph_in = Input(shape=(chunk_size, EMBEDDING_DIM))

for n_gram in ngram_filters:
    conv = Convolution1D(                                  # Layer X,   Features: 256, Kernel Size: ngram
        nb_filter=CONVOLUTION_FEATURE,                     # Number of kernels or number of filters to generate
        filter_length=n_gram,                              # Size of kernels, ngram
        activation='relu',                                 # Activation function to use
        name = str('convfilter' + str(n_gram)),            # Define length to input sequences in the first layer
        trainable=False)(graph_in)

    pool = MaxPooling1D(                                   # Layer X a,  Max Pooling: 3
        pool_length=3,                               # Define length to input sequences in the first layer
        trainable=False)(conv)                               # Size of kernels

    flat = Flatten()(pool)

    convs.append(flat)

feature_model = Sequential()

feature_model.add(Embedding(                                       # Layer 0, Start
    input_dim=nb_words + 1,                                # Size to dictionary, has to be input + 1
    output_dim=EMBEDDING_DIM,                              # Dimensions to generate
    weights=[embedding_matrix],                            # Initialize word weights
    input_length=chunk_size,                               # Define length to input sequences in the first layer
    trainable=False))                                      # Disable weight changes during training

feature_model.add(Dropout(0.25))                                   # Dropout 25%

out = Merge(mode='concat')(convs)                          # Layer 1,  Output Size: Concatted ngrams feature maps

graph = Model(input=graph_in, output=out, name = 'merge')  # Concat the ngram convolutions

feature_model.add(graph)                                           # Concat the ngram convolutions

feature_model.add(Dropout(DROP_OUT))                               # Dropout 50%

feature_model.add(Dense(                                           # Layer 3,  Output Size: 256
    output_dim=DENSE_FEATURE,                              # Output dimension
    activation='relu'))                                    # Activation function to use

In [10]:

feature_model.layers[1].set_weights(model.layers[1].get_weights())
feature_model.layers[2].set_weights(model.layers[2].get_weights())
feature_model.layers[3].set_weights(model.layers[3].get_weights())

# Compile model again (required to make predictions)
feature_model.compile(loss='categorical_crossentropy', optimizer=sgd,
                      metrics=['accuracy'])

In [11]:
pred = feature_model.predict(trainX)

In [12]:
print(pred.shape)

(4142, 256)


In [13]:
print(pred[0])

[ 0.          0.15319365  0.04749051  0.08797614  0.0032413   0.
  0.24331477  0.00792778  0.01636963  0.          0.          0.          0.
  0.          0.14415158  0.0981744   0.00125454  0.          0.
  0.03908108  0.00123329  0.          0.          0.01894081  0.05001922
  0.          0.          0.04583126  0.14853707  0.          0.14660971
  0.03657903  0.          0.09955634  0.          0.          0.14252143
  0.          0.          0.          0.          0.02539786  0.          0.
  0.          0.01437728  0.03797867  0.          0.          0.0030684   0.
  0.02532051  0.09700628  0.          0.39297104  0.          0.00466457
  0.05752011  0.          0.20336176  0.          0.030925    0.
  0.16980064  0.          0.08009711  0.19286105  0.          0.11978095
  0.03834556  0.07872675  0.01708262  0.00941921  0.          0.19684455
  0.          0.          0.          0.09044889  0.08387838  0.0328522   0.
  0.14634843  0.          0.14118324  0.          0.       

In [14]:
print(pred[4141])

[  1.01975583e-01   2.65773118e-01   0.00000000e+00   0.00000000e+00
   9.11817104e-02   0.00000000e+00   1.12901896e-01   4.43421565e-02
   9.40021276e-02   1.57443136e-01   7.95348883e-02   2.15718135e-01
   0.00000000e+00   0.00000000e+00   8.21096674e-02   4.58515473e-02
   2.73056924e-01   1.05281062e-01   1.83960628e-02   0.00000000e+00
   8.63516033e-02   5.09681217e-02   0.00000000e+00   0.00000000e+00
   4.79534231e-02   5.47050126e-02   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   8.18962883e-03   3.69190052e-02
   4.68938686e-02   0.00000000e+00   0.00000000e+00   1.52516156e-01
   0.00000000e+00   1.08909994e-01   0.00000000e+00   6.37514591e-02
   1.89529881e-01   3.63157280e-02   0.00000000e+00   2.39072338e-01
   1.50624895e-02   0.00000000e+00   3.89376506e-02   3.69260088e-02
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   9.25086737e-02   0.00000000e+00
   3.47615816e-02   2.24745125e-01